## Object detection and object counting in polygon zone
This notebook is an example how to use DeGirum PySDK to do object detection and object 
counting in polygon zone, annotating vide file.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](../../env.ini) file, located in the same directory as this notebook.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify video file name, model name, and other options here

In [ ]:
input_filename = "../../images/Traffic.mp4" # video file to process
model_name = "yolo_v5n_car_det--512x512_quant_n2x_orca1_1" # model to use

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, degirum_tools

degirum_tools.configure_colab() # configure for Google Colab

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

target = dg.CLOUD # <-- on the Cloud Platform
# target = degirum_tools.get_ai_server_hostname() # <-- on AI Server deployed in your LAN
# target = dg.LOCAL # <-- on ORCA accelerator installed on this computer

# connect to AI inference engine getting zoo URL and token from env.ini file
zoo = dg.connect(target, degirum_tools.get_cloud_zoo_url(), degirum_tools.get_token())

#### The rest of the cells below should run without any modifications

In [ ]:
import IPython
from pathlib import Path

orig_path = Path(input_filename)
ann_path = "../../workarea/" + orig_path.stem + "_annotated" + orig_path.suffix

# load model
model = zoo.load_model(model_name, overlay_line_width=1)

# define polygon zone coordinates
polygons = [
    [[265, 260], [730, 260], [870, 450], [120, 450]],
    [[400, 100], [610, 100], [690, 200], [320, 200]],
]

# create zone counter
zone_counter = degirum_tools.ZoneCounter(
    polygons, class_list=["car"], triggering_position=degirum_tools.ZoneCounter.CENTER
)

# annotate video
degirum_tools.annotate_video(model, orig_path, ann_path, zone_counter=zone_counter)


In [ ]:
# display result
IPython.display.Video(ann_path, embed=degirum_tools._in_colab())